In [1]:
import importlib
import copy
import pickle
# --------------------
from sklearn.utils import gen_batches, check_array
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.utils.validation import FLOAT_DTYPES
import numpy as np
from numpy.linalg import norm,inv,matrix_rank, pinv
# --------------------
from skopt.space import Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
# --------------------
import h5py
import datetime
import Code.SVDPLS

# import warnings
# warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [2]:
def RollingCV(tscv,X):
    cv = tscv.split(X)
    (train_index, test_index) = next(cv)
    yield(
        train_index,
        test_index
    )
    test_size = len(test_index)

    for (train_index, test_index) in (cv):
        yield(
            train_index[-test_size:],
            test_index
        )

In [3]:
with h5py.File('./data/merra2_t.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)

    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]

n_fold = 74
test_size=30
tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)

4412


In [ ]:
with h5py.File('./data/TW_PM25.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)

    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]
n_fold = 17
test_size=30
tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)

In [4]:
def dump_PLS(PLS, fn):
    np.savez(fn,
             n_comp=PLS.n_components,
             x_weights = PLS.x_weights_,
             y_weights = PLS.y_weights_,
             x_mean = PLS._x_mean,
             y_mean = PLS._y_mean
             )

def predict(dat, X, n_comp):
    X = check_array(X, copy=copy, dtype=FLOAT_DTYPES)
    x_weights = dat['x_weights'][:,:n_comp]
    y_weights = dat['y_weights'][:,:n_comp]

    coef = np.dot(x_weights, y_weights.T)
    X -= dat['x_mean']
    ypred = np.dot(X, coef)
    ypred += dat['y_mean']

    return ypred


In [5]:
print(X_train.shape)
print(Y_train.shape)

for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break

print("="*40)
for i, (train_index, test_index) in enumerate(RollingCV(tscv,X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break

(4412, 30456)
(4412, 30456)
Fold 0:
  Train: len=2192
  Test:  len=30
Fold 1:
  Train: len=2222
  Test:  len=30
Fold 2:
  Train: len=2252
  Test:  len=30
Fold 3:
  Train: len=2282
  Test:  len=30
Fold 4:
  Train: len=2312
  Test:  len=30
  Train: [   0    1    2 ... 2309 2310 2311]
  Test:  [2312 2313 2314 2315 2316 2317 2318 2319 2320 2321 2322 2323 2324 2325
 2326 2327 2328 2329 2330 2331 2332 2333 2334 2335 2336 2337 2338 2339
 2340 2341]
Fold 0:
  Train: len=2192
  Test:  len=30
Fold 1:
  Train: len=30
  Test:  len=30
Fold 2:
  Train: len=30
  Test:  len=30
Fold 3:
  Train: len=30
  Test:  len=30
Fold 4:
  Train: len=30
  Test:  len=30
  Train: [2282 2283 2284 2285 2286 2287 2288 2289 2290 2291 2292 2293 2294 2295
 2296 2297 2298 2299 2300 2301 2302 2303 2304 2305 2306 2307 2308 2309
 2310 2311]
  Test:  [2312 2313 2314 2315 2316 2317 2318 2319 2320 2321 2322 2323 2324 2325
 2326 2327 2328 2329 2330 2331 2332 2333 2334 2335 2336 2337 2338 2339
 2340 2341]


In [6]:
importlib.reload(Code.SVDPLS)
from Code.SVDPLS import ISVDPLS

n_comp_max=100
PLS = ISVDPLS(n_components=n_comp_max)
tim_tot.tic()
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    dump_PLS(PLS, f'./data/wrk/ISVDPLS_fold{i:02d}.npz')
    if (i%5==0):
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()
print(f"total time={tim_tot.tocvalue():.1f}s")

fold: 01, elapsed time: 715.2s


fold: 06, elapsed time: 49.5s


fold: 11, elapsed time: 49.3s


fold: 16, elapsed time: 49.1s


fold: 21, elapsed time: 48.9s


...

fold: 56, elapsed time: 48.8s


fold: 61, elapsed time: 49.1s


fold: 66, elapsed time: 48.9s


fold: 71, elapsed time: 48.7s


total time=1430.3s


In [7]:
PLS_list = []
for i in range(n_fold):
    PLS_list.append(np.load(f'./data/wrk/ISVDPLS_fold{i:02d}.npz'))


In [8]:
space  = [Integer(1, n_comp_max, name='n_components')]
@use_named_args(space)
def Comp_Model_Score(n_components):
    scores = np.zeros((n_fold,))
    for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
        y_true = Y_train[test_index]
        y_pred = predict(PLS_list[i], X_train[test_index], n_components)
        scores[i] = RMSE(y_true, y_pred)
    return np.mean(scores)


In [9]:
n_calls = int(np.log(n_comp_max)) + 1
print(f"n_calls = {n_calls:d}")
n_calls *= 2

tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=max(n_calls,10), 
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best param = {res_gp.x[0]:02d}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")


n_calls = 5
Iteration No: 1 started. Evaluating function at random point.


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 123.2711
Function value obtained: 7.5428
Current minimum: 7.5428
Iteration No: 2 started. Evaluating function at random point.


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 123.9730
Function value obtained: 7.5717
Current minimum: 7.5428
Iteration No: 3 started. Evaluating function at random point.


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 118.0926
Function value obtained: 7.5753
Current minimum: 7.5428
Iteration No: 4 started. Evaluating function at random point.


Iteration No: 4 ended. Evaluation done at random point.
Time taken: 121.2040
Function value obtained: 7.5717
Current minimum: 7.5428
Iteration No: 5 started. Evaluating function at random point.


...

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 119.5770
Function value obtained: 7.5242
Current minimum: 7.5242
Iteration No: 7 started. Evaluating function at random point.


Iteration No: 7 ended. Evaluation done at random point.
Time taken: 117.7950
Function value obtained: 7.5188
Current minimum: 7.5188
Iteration No: 8 started. Evaluating function at random point.


Iteration No: 8 ended. Evaluation done at random point.
Time taken: 114.8019
Function value obtained: 7.5107
Current minimum: 7.5107
Iteration No: 9 started. Evaluating function at random point.


Iteration No: 9 ended. Evaluation done at random point.
Time taken: 119.5022
Function value obtained: 7.5177
Current minimum: 7.5107
Iteration No: 10 started. Evaluating function at random point.


Iteration No: 10 ended. Evaluation done at random point.
Time taken: 116.1804
Function value obtained: 7.5306
Current minimum: 7.5107
----------------------------------------
Best param = 07
Best score = 7.5107
total time = 1191.8s


In [10]:
n_calls = int(np.log(n_comp_max)) + 1
print(f"n_calls = {n_calls:d}")
n_calls *= 2

tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=max(n_calls,20), 
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best param = {res_gp.x[0]:02d}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")


n_calls = 5
Iteration No: 1 started. Evaluating function at random point.


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 118.3513
Function value obtained: 7.5428
Current minimum: 7.5428
Iteration No: 2 started. Evaluating function at random point.


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 122.4469
Function value obtained: 7.5717
Current minimum: 7.5428
Iteration No: 3 started. Evaluating function at random point.


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 117.4583
Function value obtained: 7.5753
Current minimum: 7.5428
Iteration No: 4 started. Evaluating function at random point.


Iteration No: 4 ended. Evaluation done at random point.
Time taken: 119.7760
Function value obtained: 7.5717
Current minimum: 7.5428
Iteration No: 5 started. Evaluating function at random point.


...

Iteration No: 18 ended. Search finished for the next optimal point.
Time taken: 119.2242
Function value obtained: 7.5493
Current minimum: 3.3942
Iteration No: 19 started. Searching for the next optimal point.


/home/ubuntu/miniconda3/envs/pls/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(100)] before, using random point [np.int64(24)]
  warnings.warn(


Iteration No: 19 ended. Search finished for the next optimal point.
Time taken: 119.5869
Function value obtained: 7.5157
Current minimum: 3.3942
Iteration No: 20 started. Searching for the next optimal point.


/home/ubuntu/miniconda3/envs/pls/lib/python3.11/site-packages/skopt/optimizer/optimizer.py:517: UserWarning: The objective has been evaluated at point [np.int64(100)] before, using random point [np.int64(14)]
  warnings.warn(


Iteration No: 20 ended. Search finished for the next optimal point.
Time taken: 119.3726
Function value obtained: 7.5121
Current minimum: 3.3942
----------------------------------------
Best param = 100
Best score = 3.3942
total time = 2356.3s
